In [109]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import pandas as pd
import scipy.linalg as la

import seaborn as sn
import matplotlib.font_manager as fm

In [110]:
# --- Design plot setup --- #

font_path = "/Users/pedrobraga/Documents/Cambridge/wa_two/cmunci.ttf"
custom_font = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = custom_font.get_name()
sn.set(font=custom_font.get_name())

base = {
    "axes.facecolor": "white",
    "axes.edgecolor": "black",
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.color": "black",
    "ytick.color": "black",
    "xtick.bottom": True,
    "xtick.top": True,
    "ytick.left": True,
    "ytick.right": True,
}

sn.set_theme(context='notebook', style=base, palette='deep', color_codes=True, rc=None)

In [111]:
# --- physical constants --- #

hbar = 0.658 # ev fs
m0 = 5.6856800 # fs{2} eV nm{-2}
veps0 = 5.52638e-2 # c{2} eV{-1} nm{-1}
n = 1.82
c0 = 299.792 # nm/fs
kb = 8.617e-5 # ev K {-1}

# --- system parameters --- #

Lwell = 0.636 # nm
A = Lwell**2 # nm{2}
Gamma0 = 0.0067 # eV
Msigma = 8 # fs ev nm{-1}
DAC = 1.9 # eV
DOP = 158 # ev nm{-1}
EgZero = 2.565 # eV

alphaOne = -1e-4 # material specific constant for Varshni shift
alphaTwo = 250 # material specific constant for Varshni shift

# --- simulation parameters --- #

k_max = .75
N = 500
dk = k_max/N

k_vals = np.linspace(dk, k_max, N)

In [112]:
# --- import eigenvalues and eigenvectors calculated by solving the Wannier equation --- #

eigenvalues = np.loadtxt('Eigenvalues5.txt')
eigenvectors = np.loadtxt('Eigenvectors5.txt')
polarEigenvectors = np.loadtxt('EigenvectorsSlashed5.txt')

In [136]:
def BandGap(temperature):
    return EgZero - (alphaOne*temperature**2)/(alphaTwo+temperature)

def FFT(mu, polarEigenvectors): # polar coordinate transfor proptp the fourier transform to real space evaluated at r = 0

    integral = np.sum(polarEigenvectors[mu] * A * dk * k_vals/(2 * np.pi))
    return np.absolute(integral)**2

def RadiativeRecom(mu, eigenvalues, eigenvectors, T): # Temperature independent radiactive recombination of eletrons

    E0 = BandGap(T) - eigenvalues[mu]

    wavefunction = FFT(mu, polarEigenvectors)
    C = (hbar * Msigma**2) / (2 * m0**2 * veps0 * n * c0)
    return C * wavefunction / E0

def ExcitonDephase(mu, temp): # Phenomenological/experimental non-radiative dephasing term

    return RadiativeRecom(mu, eigenvalues, eigenvectors, temp)*temp + (0.0067 / (np.exp(0.037/(kb*temp))-1))

In [114]:
def Absorption(energy_range, eigenvalues, eigenvectors, temp):

    alpha = []

    for E in energy_range:
        alphaMu = 0

        for mu in range(len(eigenvalues)):

            E0 = BandGap(temp) - eigenvalues[mu]

            numerator = 2 * RadiativeRecom(mu, eigenvalues, polarEigenvectors, temp) * ExcitonDephase(mu, temp)
            denominator = (E0 - E)**2 + (ExcitonDephase(mu, temp) + RadiativeRecom(mu, eigenvalues, polarEigenvectors, temp))**2
            alphaMu += numerator/denominator

        alpha.append(alphaMu)

    return alpha

In [134]:
def FinalPlot(temps):

    E = np.linspace(2, 3, 10000)
    data = {"Energy": E}
    norm = 0
    
    for T in temps:
        data[f'{T}'] = Absorption(energy_range=E, eigenvalues=eigenvalues, eigenvectors=polarEigenvectors, temp=T)
        
        maximum = np.max(data[f'{T}'])
        if maximum > norm:
            norm = maximum

    df = pd.DataFrame(data)

    mosaic = """ 
            AA
            BC
            
            """
    fig = plt.figure(layout='constrained', figsize=(8,8))
    ax_dict = fig.subplot_mosaic(mosaic, gridspec_kw={'hspace': 0})

    for column in df.columns[1:]:
        ax_dict["A"].plot((df['Energy'] - eigenvalues[0]), df[column]/norm, label=f'T = {column} K', linewidth=3)
        ax_dict["A"].set_xlim(2.15, 2.45)
        ax_dict["A"].set_title('(a)',  fontsize=16)
        ax_dict["A"].legend(fontsize=16)
        ax_dict["A"].tick_params(labelsize=16)

        ax_dict["B"].plot((df['Energy'] - eigenvalues[0]), df[column]/norm, label=f'T = {column} K', linewidth=3)
        ax_dict["B"].set_xlim(2.2, 2.3)
        ax_dict["B"].set_title('(b)',  fontsize=16)
        ax_dict["B"].tick_params(labelsize=16)

        ax_dict["B"].text(2.24, 0.9, '1s', color='#4C72B0', fontsize=16, ha='center')
        ax_dict["B"].text(2.25, 0.18, '1s', color='#DD8452', fontsize=16, ha='center')
        ax_dict["B"].text(2.27, 0.1, '1s', color='#55A868', fontsize=16, ha='center')

        ax_dict["B"].xaxis.set_major_locator(ticker.MultipleLocator(0.05))

        ax_dict["C"].plot((df['Energy'] - eigenvalues[0]), df[column]/norm, label=f'T = {column} K', linewidth=3)
        ax_dict["C"].set_xlim(2.35, 2.45)
        ax_dict["C"].set_ylim(-.005, 0.05)
        ax_dict["C"].set_title('(c)', fontsize=16)
        ax_dict["C"].tick_params(labelsize=16)

        ax_dict["C"].text(2.37, 0.04, '2s', color='#4C72B0', fontsize=16, ha='center')
        ax_dict["C"].text(2.38, 0.01, '2s', color='#DD8452', fontsize=16, ha='center')
        ax_dict["C"].text(2.40, 0.005, '2s', color='#55A868', fontsize=16, ha='center')

        ax_dict["C"].text(2.388, 0.006, '3s', color='#4C72B0', fontsize=16, ha='center')
        ax_dict["C"].text(2.42, 0.003, '3s', color='#55A868', fontsize=16, ha='center')

        fig.supxlabel("$E - E_{1s}$ (eV)", fontsize=16)
        fig.supylabel("Absorption coefficient (normalised)", fontsize=16)

        plt.savefig('absorption.pdf', dpi=600)


In [118]:
def PlotLinewidth(eigenvalues, Ti, Tf):

    temps = np.linspace(Ti, Tf, 1000)
    
    Rad0 = []
    NonRad0 = []
    Total0 = []
    Rad1 = []
    NonRad1 = []
    Total1 = []
    Rad2 = []
    NonRad2 = []
    Total2 = []

    for i in [0, 1, 2]:
        for T in temps:

            if i == 0:
                gamma = RadiativeRecom(i, eigenvalues, eigenvectors)
                Gamma = ExcitonDephase(i, T)
                Rad0.append(1000 * gamma)
                NonRad0.append(1000 * Gamma)
                Total0.append(1000*gamma + 1000*Gamma)

            elif i == 1:
                gamma = RadiativeRecom(i, eigenvalues, eigenvectors)
                Gamma = ExcitonDephase(i, T)
                Rad1.append(1000*gamma)
                NonRad1.append(1000*Gamma)
                Total1.append(1000*gamma + 1000*Gamma)

            elif i == 2:
                gamma = RadiativeRecom(i, eigenvalues, eigenvectors)
                Gamma = ExcitonDephase(i, T)
                Rad2.append(1000*gamma)
                NonRad2.append(1000*Gamma)
                Total2.append(1000*gamma + 1000*Gamma)
    
    fig = plt.figure(layout='constrained', figsize=(5, 10))
    gs = fig.subplots(3, 1, sharex=True)
    
    gs[0].set_title('1s - (a)', fontsize=16)
    gs[0].plot(temps, Total0, label='Total', linewidth=3)
    gs[0].plot(temps, NonRad0, label='Phonon', linewidth=3)
    gs[0].plot(temps, Rad0, label='Radiative', linewidth=3)
    
    gs[0].fill_between(temps, Total0, alpha=0.4)
    gs[0].fill_between(temps, NonRad0, alpha=0.4)
    gs[0].fill_between(temps, Rad0, alpha=0.4)
    gs[0].tick_params(labelsize=16)
    gs[0].set_ylim(0, 5)
    gs[0].set_xlim(0, 300)
    
    gs[1].set_title('2s - (b)', fontsize=16)
    gs[1].plot(temps, Total1, label='Total', linewidth=3)
    gs[1].plot(temps, NonRad1, label='Non-radiative', linewidth=3)
    gs[1].plot(temps, Rad1, label='Radiative', linewidth=3)

    gs[1].fill_between(temps, Total1, alpha=0.4)
    gs[1].fill_between(temps, NonRad1, alpha=0.4)
    gs[1].fill_between(temps, Rad1, alpha=0.4)
    gs[1].set_ylabel('Exciton linewidth (meV)', fontsize=16)
    gs[1].tick_params(labelsize=16)
    gs[1].set_xlim(0, 300)
    gs[1].set_ylim(0, 5)
    gs[1].legend(loc='upper left', fontsize=16)

    gs[2].set_title('3s - (c)', fontsize=16)
    gs[2].plot(temps, Total2, label='Total', linewidth=3)
    gs[2].plot(temps, NonRad2, label='Non-radiative', linewidth=3)
    gs[2].plot(temps, Rad2, label='Radiative', linewidth=3)

    gs[2].fill_between(temps, Total2, alpha=0.4)
    gs[2].fill_between(temps, NonRad2, alpha=0.4)
    gs[2].fill_between(temps, Rad2, alpha=0.4)
    gs[2].set_xlabel('Temperature (K)', fontsize=16)
    gs[2].tick_params(labelsize=16)
    gs[2].set_xlim(0, 300)
    gs[2].set_ylim(0, 5)

    plt.savefig('linewidth.pdf', dpi = 600)